In [168]:
import pandas as pd
import numpy as np
import string
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,roc_auc_score
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import xgboost as xgb
from imblearn.ensemble import BalancedBaggingClassifier
from imblearn.under_sampling import RandomUnderSampler
from sklearn.linear_model import LinearRegression,LogisticRegression
from collections import Counter
from imblearn.combine import SMOTEENN


In [8]:
train1=pd.read_csv('/Users/hemanth/Downloads/News/Train/Train_1.csv')
train2=pd.read_csv('/Users/hemanth/Downloads/News/Train/Train_2.csv')
train3=pd.read_csv('/Users/hemanth/Downloads/News/Train/Train_3.csv')
train4=pd.read_csv('/Users/hemanth/Downloads/News/Train/Train_4.csv')
train5=pd.read_csv('/Users/hemanth/Downloads/News/Train/Train_5.csv')
train6=pd.read_csv('/Users/hemanth/Downloads/News/Train/Train_6.csv')
train7=pd.read_csv('/Users/hemanth/Downloads/News/Train/Train_7.csv')
train8=pd.read_csv('/Users/hemanth/Downloads/News/Train/Train_8.csv')
train9=pd.read_csv('/Users/hemanth/Downloads/News/Train/Train_9.csv')
train10=pd.read_csv('/Users/hemanth/Downloads/News/Train/Train_10.csv')
train=pd.concat([train1,train2,train3,train4,train5,train6,train7,train8,train9,train10])

In [11]:
df1=pd.read_excel("/Users/hemanth/Downloads/News/Item_Category_Map/Item_Category_Map_1.xlsx")
df2=pd.read_excel("/Users/hemanth/Downloads/News/Item_Category_Map/Item_Category_Map_2.xlsx")
cat=pd.concat([df1,df2])

In [82]:
df1=pd.read_csv("/Users/hemanth/Downloads/News/Item_Attributes/Item_Attributes_1.csv")
df2=pd.read_csv("/Users/hemanth/Downloads/News/Item_Attributes/Item_Attributes_2.csv")
df3=pd.read_csv("/Users/hemanth/Downloads/News/Item_Attributes/Item_Attributes_3.csv")
att=pd.concat([df1,df2,df3])

In [9]:
test=pd.read_csv("/Users/hemanth/Downloads/News/test-data.csv")

In [78]:
imp=pd.read_csv("/Users/hemanth/Downloads/News/imp.csv")

/Users/hemanth/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [1]:
df=pd.read_csv("/Users/hemanth/Downloads/News/df.csv")

In [3]:
df_test=pd.read_csv("/Users/hemanth/Downloads/News/df_test.csv")

In [132]:
import sys
reload(sys)
sys.setdefaultencoding('utf-8')

att=att["item_id|title|description|timestamp_creation"]
li=att.astype("str").str.split("|")

l1=[]
for i in li:
    if len(i)>0:
        l1.append(i[0])
    else:
        l1.append(np.nan)

l2=[]
for i in li:
    if len(i)>1:
        l2.append(i[1])
    else:
        l2.append(np.nan)
        
df=pd.DataFrame()
df["item_id"]=pd.Series(l1)
df["col1"]=pd.Series(l2)

df["len"]=df["item_id"].astype("str").str.len()
li=df["len"]==8
df=df[li]
df.drop(["len"],axis=1,inplace=True)

gh=pd.concat([pd.Series(row['item_id'],str(row['col1']).split(' ')) for _, row in df.iterrows()]).reset_index()
gh.columns=[["word","item_id"]]
gh["word"]=gh['word'].map(lambda x: x.lstrip('1234567890').rstrip('1234567890'))

from nltk.corpus import stopwords
stop=stopwords.words("english")

gh["word"]=gh['word'].str.lower()
gh["word"]=gh["word"].apply(lambda x: x if x not in stop else np.nan)
gh=gh.dropna()

gh=gh.reset_index(drop=True)

l=[]
for i in gh["word"]:
    l.append(i)
    
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

lis=[]
for i in l:
    lis.append(tokenizer.tokenize(i))

lk=[]
for i in lis:
    if len(i)>0:
        lk.append(i[0])
    else:
        lk.append(np.nan)

gh["word"]=pd.Series(lk)

item=pd.concat([gh,word])
item=item.drop_duplicates()
item=item.dropna()
item=item.reset_index(drop=True)

li=[]
for i in item["word"]:
    i = "".join(l for l in i if l not in string.punctuation)
    li.append(i)
item["word"]=pd.Series(li)


item["len"]=item["word"].str.len()

k=item['word'].map(lambda x: x.lstrip('10000000‘'))
item["word"]=pd.Series(k)

li=item["word"]=="2"
item=item[~li]
li=item["word"]==""
item=item[~li]

ch=item.groupby(["word"],as_index=False).count()
ch.columns=[["word","num","num2"]]
item2=pd.merge(item,ch[["word","num"]],on="word",how="left")
li=item2["num"]>1
item2=item2[li]

li=item2["len"]==2
item2=item2[~li]

item2.drop(["len","num"],axis=1,inplace=True)
item2=item2.reset_index(drop=True)

item2.to_csv("/Users/hemanth/Downloads/News/word.csv",index=False)

In [431]:
item=pd.read_csv("/Users/hemanth/Downloads/News/word.csv")

In [441]:
pd.Series(item["item_id"].value_counts()>8).value_counts()

False    31996
True     12223
Name: item_id, dtype: int64

In [6]:
ch=df_test.copy()

In [16]:
ch=ch[["impression_id","item_id"]]

In [17]:
ch["click"]=np.zeros(len(ch))

/Users/hemanth/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [18]:
check=pd.merge(test,ch,on=["impression_id","item_id"],how="left")

In [20]:
li=check["click"].isnull()

In [21]:
check=check[li]

In [27]:
item=df[["item_id","per_item"]]

In [ ]:
check=pd.merge(check,item,on=["item_id"],how="left")

In [11]:
train=pd.read_csv('/Users/hemanth/Downloads/df.csv')

/Users/hemanth/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [14]:
train.drop(["impression_id","refrenceUrl","time","hour","timestamp_impression","Day","item_id","uuid","uvh","geo","site_id","adunit_id",'per_uuid','no_uuid'],axis=1,inplace=True)

In [17]:
train=train.fillna(0)

In [137]:
test=test.fillna(0)

In [68]:
train.drop(['click'],axis=1,inplace=True)

In [69]:
X_train,X_test,y_train,y_test=train_test_split(train.values,Y,test_size=0.3)

In [48]:
model = Sequential()
model.add(Dense(60, input_dim=6, kernel_initializer='normal', activation='relu'))
# model.add(Dense(30, kernel_initializer='normal', activation='relu'))
model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['mean_squared_error'])

In [49]:
model.fit(X_train,y_train,epochs=1,batch_size=256,class_weight={1:1,2:5})

Epoch 1/1
12602429/12602429 [==============================] - 164s - loss: -1.3866 - mean_squared_error: 0.0181   


In [50]:
preds=model.predict(X_test,batch_size=256)

In [51]:
preds=preds.reshape(1,-1)

In [67]:
Y=train['click']
Y[Y==2]=0

/Users/hemanth/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [94]:
test=pd.read_csv("/Users/hemanth/Downloads/df_test.csv")

In [134]:
test.drop(["impression_id","refrenceUrl","time","hour","timestamp_impression","Day","item_id","uuid","uvh","geo","site_id","adunit_id",'per_uuid','no_uuid'],axis=1,inplace=True)

In [112]:
params={
    'objective':'multi:softprob',
    'num_class':2,
    'eta':0.1,
}

# w=np.zeros(len(Y))
# w[Y==0]=3
# w[Y==1]=1

w1=np.zeros(len(y_train))
w1[y_train==0]=55
w1[y_train==1]=1

def run(X,Y,test,params,w):
    dtrain=xgb.DMatrix(X,label=Y,weight=w)
    dtest=xgb.DMatrix(test)
    bst=xgb.train(params,dtrain)
    prods=bst.predict(dtest)
    return prods

In [151]:
preds=run(X_resampled, y_resampled,X_test,params,w1)

In [155]:
preds1=preds[:,1]

In [156]:
roc_auc_score(y_test,preds1)

0.82891702372993281

In [157]:
preds2=(preds1>0.5).astype('int')

In [158]:
print accuracy_score(y_test,preds2)
print confusion_matrix(y_test,preds2)

0.703439447425
[[  75745   19133]
 [1582603 3723561]]


In [111]:
print accuracy_score(y_test,preds2)
print confusion_matrix(y_test,preds2)

0.702668114782
[[  76207   18671]
 [1587231 3718933]]


In [130]:
bbc = BalancedBaggingClassifier(base_estimator=LogisticRegression())
# bbc.fit(X_train,y_train)

In [122]:
pred=bbc.predict_proba(X_test)

In [124]:
pred1=pred[:,1]

In [125]:
pred1

array([ 0.78429974,  0.57356926,  0.09791792, ...,  0.48422523,
        0.78668007,  0.69088311])

In [126]:
roc_auc_score(y_test,pred1)

0.80195165751778297

In [127]:
pred2=(pred1>0.5).astype('int')

In [129]:
print accuracy_score(y_test,pred2)
print confusion_matrix(y_test,pred2)

0.747841805341
[[  65370   29508]
 [1332409 3973755]]


In [131]:
bbc.fit(train.values,Y.values)

BalancedBaggingClassifier(base_estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
             bootstrap=True, bootstrap_features=False, max_features=1.0,
             max_samples=1.0, n_estimators=10, n_jobs=1, oob_score=False,
             random_state=None, ratio='auto', replacement=False, verbose=0,
             warm_start=False)

In [132]:
test=pd.read_csv('/Users/hemanth/Downloads/df_test.csv')

In [140]:
f=bbc.predict(test.values)

In [141]:
pd.Series(f).value_counts()

0    156539
1    138208
dtype: int64

In [166]:
f=pd.Series(f)

In [167]:
f.to_csv("/Users/hemanth/Desktop/bblr1.csv",index=False)

In [ ]:
sm = SMOTEENN()
X_resampled, y_resampled = sm.fit_sample(X_train, y_train)

In [161]:
rus = RandomUnderSampler(random_state=0)
X_res, y_res = rus.fit_sample(train.values, Y.values)

In [163]:
print(sorted(Counter(y_resampled).items()))
print(sorted(Counter(y_train).items()))

[(0, 316179), (1, 316179)]
[(0, 316179), (1, 17687292)]


In [159]:
lr=LogisticRegression()
lr.fit(X_resampled, y_resampled)
preds=lr.predict(X_test)

In [160]:
print accuracy_score(y_test,preds)
print confusion_matrix(y_test,preds)

0.748521489002
[[  65312   29566]
 [1328680 3977484]]


In [164]:
lr=LogisticRegression()
lr.fit(X_res,y_res)
f=lr.predict(test.values)